#Next Generation Sequence Analysis

##Overview

The analysis of next generation sequence data typically involves the subsequent application of multiple tools and utilities, each with their own specific set of parameters and usage characteristics. Although there are currently many different software packages that can be used at various stages of analysis, and are often dictated by the type of sequencing conducted, the general analytical pipeline can be described as follows:

<img src="ngs_pipeline.png" alt="Pipeline" style="width: 600px;"/>

We will be using sequences generated from an Illumina HiSeq2000 machine today, and as such the rest of this document will pertain to analysis of this type of sequencing data.

**Quality Control**   
The first step of any sequencing project is an assessment of the raw sequence data that were generated. There are a number of simple ways to check your sequences. One makes use of the expectation that each sequence position should have a consistent nucleotide usage across all sequence reads, while another assesses per-base quality scores. Often times Illumina sequence reads need to be trimmed at their ends. 

**Alignment**   
After assessing the quality of the sequence and filtering/trimming as needed, the next step is to align the sequence against a reference. There are many software packages for doing this, and most make use of a Burrows-Wheeler transformation to speed up this process. The resulting alignments are then described in SAM format, which stores various information about the sequences including alignment position and mapping quality. More information can be found here: http://genome.sph.umich.edu/wiki/SAM

**Optimization**   
Once a sequence has been aligned, there are a number of additional steps that should be untaken. Some of these, such as cleaning and sorting the SAM file, are required in order to prepare the file for further software packages. Others are tailored to correct the alignments for artifacts stemming from both machine and sequence-based characteristics such as alignment problems around INDELs and quality scores affected by sequencing cycle.

**Variant Calling**   
The accurate identification of genetic variation, both within and between genomic sequences, is one of the primary goals of next generation sequence analysis. Different software packages achieve this in various ways, and range from identifying a variant from a single sequence (heterozygous, or homozygous but different from reference) to across a population of sequences. Quality scores of individual bases are also typically taken into account to minimize false positive results.

**Variant Interpretation**   
The next step after identifying variation is the interpretation of what potential functional role, if any, that it may have. For single nucleotide variants, this is often an assessment of whether a particular allele would result in an amino acid change (non-synonymous change) or a premature stop codon (non-sense change), if found in the coding region of a gene. Other features can include whether a splice site is created/destroyed, or if a variant falls within a known functional region (e.g. transcription factor binding site).

Today we will be taking a set of genomic sequences through our own pipeline. These sequences were taken from the 1000 Genomes Project (sample NA12891) that sequenced to 100X coverage on an Illumina HiSeq2000 machine at the Broad Institute. We will also be using the personal genome sequence of Gil Omenn, M.D., Ph.D. (http://www.ccmb.med.umich.edu/omenn), a member of our department, that was generated as part of the initial 100 Pioneers of the 100K Wellness Project (https://www.systemsbiology.org/research/100k-wellness-project/). In the interests of time, we will only be analyzing a small subset of these sequences taken from a 100 Kbp region on chromosome 21. 

##iPython Notebooks
We will be making use of iPython Notebooks for today's lab session. These allow you to both comment and actively run python or shell code sequentially and view the results in the browser itself. Indeed, this is what you are viewing now! Today, we will be running many third-party software platforms and as such will need to prepend a "!" character before our commands so it knows to run it as in the shell. Alternatively, these commands (without the "!") can be copy/pasted into a terminal window and run there instead. We will start by viewing the files that were provided.

In [ ]:
! ls

These files include a reference genome (ref.fa) and associated alignment indexes for bowtie2 (.bt2) and bwa (.bwt). We also see some sequence files (e.g. s_3_1_sequence.txt) as well as a variance call file for dbsnp.

Let's take a quick look at one of the sequence files to see what it looks like:

In [ ]:
! head -20 s_3_1_sequence.txt

###Assessment of the quality of raw fastq sequence files. 
Paired-end sequencing data files are typically formatted in an s\_[lane]\_[pair]\_sequence.txt file, where lane indicates the lane in which the sequence was obtained from the flow cell, and pair is an index indicating that the sequences in \_1\_ are paired with sequences in \_2\_. In this directory, lane 3 corresponds to sample NA12891 while lane 8 corresponds to Dr. Omenn's genome. Check the quality of all sequences at once with the following command:



In [ ]:
! /class/local/bin/fastqc s_3_1_sequence.txt s_3_2_sequence.txt s_8_1_sequence.txt s_8_2_sequence.txt

This will create a directory for each fastq file appended with “\_fastqc”, inside of which is a file named fastqc_data.txt. You can view these files with the more (or less) command from the terminal. Alternatively, you can launch firefox and view the fastqc_report.html file that includes images along with the data. Or, as we will do here, you can view directly in this notebook with a little amount of custom syntax:

In [ ]:
from IPython.display import HTML
HTML('<iframe src=./s_3_1_sequence_fastqc.html width=1200 height=700></iframe>')

In [ ]:
HTML('<iframe src=./s_3_2_sequence_fastqc.html width=1200 height=700></iframe>')

In [ ]:
HTML('<iframe src=./s_8_1_sequence_fastqc.html width=1200 height=700></iframe>')

In [ ]:
HTML('<iframe src=./s_8_2_sequence_fastqc.html width=1200 height=700></iframe>')

Do any of the sequences fail the “Basic Statistics”? Which individual modules report a failure in quality?

###Alignment of sequences to reference
Next, we will run BWA to align the sequences to the reference. There are two steps to a BWA alignment: the aln step, which creates the suffix array from the input sequences, and the sampe step which uses those arrays to do paired end alignment. 

In [ ]:
! /class/local/bin/bwa aln ref.fa s_3_1_sequence.txt > s_3_1_sequence.sai
! /class/local/bin/bwa aln ref.fa s_3_2_sequence.txt > s_3_2_sequence.sai
! /class/local/bin/bwa sampe ref.fa s_3_1_sequence.sai s_3_2_sequence.sai s_3_1_sequence.txt s_3_2_sequence.txt > s_3.sam

Now, convert the SAM files to binary BAM format using samtools

In [ ]:
! /class/local/bin/samtools view -bhS s_3.sam > s_3.bam

We can also use samtools to summarize the aligned sequences

In [ ]:
! /class/local/bin/samtools flagstat s_3.bam

Repeat this for the lane 8 sequences in the sections below. 

In [ ]:
! /class/local/bin/bwa aln ref.fa s_8_1_sequence.txt > s_8_1_sequence.sai
! /class/local/bin/bwa aln ref.fa s_8_2_sequence.txt > s_8_2_sequence.sai
! /class/local/bin/bwa sampe ref.fa s_8_1_sequence.sai s_8_2_sequence.sai s_8_1_sequence.txt s_8_2_sequence.txt > s_8.sam

In [ ]:
! /class/local/bin/samtools view -bhS s_8.sam > s_8.bam

In [ ]:
! /class/local/bin/samtools flagstat s_8.bam

What percentage of reads was successfully mapped? Is this consistent between the samples? 

Now that we have assessed this, we can filter out all unmapped reads using samtools and the associated flags found in the alignments

In [ ]:
! /class/local/bin/samtools view -bh -F 0x4 s_3.bam > s_3_mapped.bam
! /class/local/bin/samtools view -bh -F 0x4 s_8.bam > s_8_mapped.bam

Why would we want to filter out unmapped reads? Furthermore, what types of sequence analysis would we not want to filter these out?

###Processing alignments using Picard
Picard is a collection of useful utilities for manipulating SAM/BAM files. We will first start out by adding Read Groups to our files. Read Groups help categorize and organize sets of sequences. We will simultaneously sort the file by coordinate, which is a requirement for later analysis.

In [ ]:
! java -jar /class/local/bin/picard/AddOrReplaceReadGroups.jar I=s_3_mapped.bam O=NA12891_sorted.bam SO=coordinate ID=FLOWCELL1.LANE3 PL=ILLUMINA LB=LIB_101_PE PU=NULL SM=NA12891 CN=BROAD
! java -jar /class/local/bin/picard/AddOrReplaceReadGroups.jar I=s_8_mapped.bam O=Omenn_sorted.bam SO=coordinate ID=FLOWCELL1.LANE8 PL=ILLUMINA LB=LIB_101_PE PU=NULL SM=Omenn CN=NYGENOME


Sequences often contain PCR or optical duplicates. These can be easily flagged using Picard.

In [ ]:
! java -jar /class/local/bin/picard/MarkDuplicates.jar I=NA12891_sorted.bam O=NA12891_sorted_dedup.bam M=NA12891_sorted_dedup.metrics
! java -jar /class/local/bin/picard/MarkDuplicates.jar I=Omenn_sorted.bam O=Omenn_sorted_dedup.bam M=Omenn_sorted_dedup.metrics

How many duplicates were found for each sample? 

###Calling genomic variation using GATK
Now that the files are sorted by coordinates, can you figure out approximately what region on chromosome 21 was targeted for analysis? Hint: You can use the unix commands ‘head –NNN’ and ‘tail –n NNN to display the first and last NNN lines of a file, respectively.


In [ ]:
! samtools view NA12891_sorted.bam | head -1
! samtools view NA12891_sorted.bam | tail -n 1

Next, we will optimize our alignments with GATK. The Genome Analysis ToolKit (GATK) has two major components that we will use here: realignment and recalibration. First, however, we will need to index these files using samtools.

In [ ]:
! /class/local/bin/samtools index NA12891_sorted_dedup.bam
! /class/local/bin/samtools index Omenn_sorted_dedup.bam

Next, we can use GATK to create a set of targets of known (dbsnp.vcf) or possibly novel indels from the sequences themselves using the RealignerTargetCreator walker.

In [ ]:
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -T RealignerTargetCreator -R ref.fa -I NA12891_sorted_dedup.bam -I Omenn_sorted_dedup.bam -o output.intervals -known dbsnp.vcf

These will be used as input in the IndelRealigner walker.

In [ ]:
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -I NA12891_sorted_dedup.bam -R ref.fa -T IndelRealigner -targetIntervals output.intervals -o NA12891_sorted_dedup_realign.bam -known dbsnp.vcf

! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -I Omenn_sorted_dedup.bam -R ref.fa -T IndelRealigner -targetIntervals output.intervals -o Omenn_sorted_dedup_realign.bam -known dbsnp.vcf


We can now use the realigned files to create a recalibration report.

In [ ]:
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -T BaseRecalibrator -I NA12891_sorted_dedup_realign.bam -I Omenn_sorted_dedup_realign.bam -R ref.fa -knownSites dbsnp.vcf -o recalibration_report.grp

Finally, we can use the recalibration report to generate new base qualities for each sequence

In [ ]:
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -T PrintReads -R ref.fa -I NA12891_sorted_dedup_realign.bam -BQSR recalibration_report.grp -o NA12891.bam
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -T PrintReads -R ref.fa -I Omenn_sorted_dedup_realign.bam -BQSR recalibration_report.grp -o Omenn.bam


Now that we have sorted, cleaned and optimized alignment files, we can now call SNP and INDEL variants using the UnifiedGenotyper. 

In [ ]:
! java -jar /class/local/bin/GenomeAnalysisTK/GenomeAnalysisTK.jar -T UnifiedGenotyper -R ref.fa -I NA12891.bam -I Omenn.bam --dbsnp dbsnp.vcf -o snps.raw.vcf

This will output a file in VCF format (see http://vcftools.sourceforge.net/specs.html), which is a text file and can thus be looked at using the more command. The last two columns will report information for each individual sample (NA12891, Omenn). How many total variants were identified? Earlier you were asked to estimate from the alignments what region on chromosome 21 was targeted. Using the estimated size of this region, what is the average density of snps (i.e. X SNPs per Y base pairs). How does this compare to known estimates?

In [ ]:
! head -50 snps.raw.vcf

###Assessment and annotation of predicted variation
We can take our variant calls and investigator whether any might be functionally relevant. There are many different software packages for doing this (e.g. ANNOVAR, VAAST), however today we will make use of of the Ensembl Variant Effect Predictor (VEP).

- open firefox, navigate to http://grch37.ensembl.org/Homo_sapiens/Tools/VEP
- at the top, select “New VEP job”
- set “Input file format” to “VCF” and click on “Browse” to select the snp file you generated on the previous step
- under “Transcript database to use”, select “Gencode basic transcripts”
- click on “Run”
- when done processing, click on “View results”

How many SNPs fell into coding regions, and what genes were affected? How many were synonymous vs non-synonymous? How do they compare between samples? For the non-synonymous variants (if any), would you consider the amino acid changes to be significant? 

###Comparison of alignment approaches
If there is time, go to the top of this browser page and select File -> Make a Copy to open a new cloned version of this workflow. Alter this new version to use bowtie2 instead of BWA. The syntax for bowtie2 is as follows:

/class/local/bin/bowtie2 ref -1 s_3_1_sequence.txt -2 s_3_2_sequence.txt -S s_3.sam   
/class/local/bin/bowtie2 ref -1 s_8_1_sequence.txt -2 s_8_2_sequence.txt -S s_8.sam   
 
Do you observe any differences in the variation that is detected, specifically with regards to coding consequences? How would this affect your choice of sequence aligners?

###Software used in this session

####FASTQC:		http://www.bioinformatics.babraham.ac.uk/projects/fastqc/FastQC 
FASTQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis. The main functions of FastQC are 
-	Import of data from BAM, SAM or FastQ files (any variant)
-	Providing a quick overview to tell you in which areas there may be problems
-	Summary graphs and tables to quickly assess your data
-	Export of results to an HTML based permanent report
-	Offline operation to allow automated generation of reports without running the interactive application

####BWA: 			http://sourceforge.net/projects/bio-bwa/
BWA is a program for aligning sequencing reads against a large reference genome (e.g. human genome). It has two major components, one for read shorter than 150bp and the other for longer reads.
-	Support Illumina, SOLiD, 454, Sanger reads, PacBio reads and assembly contigs
-	Gapped alignment and paired-end mapping
-	Accurate, fast and lightweight

####BOWTIE2:	http://bowtie-bio.sourceforge.net/bowtie2/index.shtml
Bowtie2 is another program for aligning DNA sequencing reads, and is part of the Tuxedo Suite software package that is widely used for analyzing RNA sequences as well.  It has many of the same attributes as BWA (gapped, paired-end mapping, soft clipping of reads, etc) and shows a similar mapping accuracy for a slightly improved computational runtime.

####GATK: 		http://www.broadinstitute.org/gatk/
The Genome Analysis Toolkit or GATK is a software package developed at the Broad Institute to analyse next-generation resequencing data. The toolkit offers a wide variety of tools, with a primary focus on variant discovery and genotyping as well as strong emphasis on data quality assurance. Its robust architecture, powerful processing engine and high-performance computing features make it capable of taking on projects of any size.

####PICARD: 		http://picard.sourceforge.net/
Picard comprises Java-based command-line utilities that manipulate SAM files, and a Java API (SAM-JDK) for creating new programs that read and write SAM files. Both SAM text format and SAM binary (BAM) format are supported.

####SAMTOOLS: 		http://samtools.sourceforge.net/
SAM (Sequence Alignment/Map) format is a generic format for storing large nucleotide sequence alignments. SAM Tools provide various utilities for manipulating alignments in the SAM format, including sorting, merging, indexing and generating alignments in a per-position format. SAM aims to be a format that: 
-	Is flexible enough to store all the alignment information generated by various alignment programs; 
-	Is simple enough to be easily generated by alignment programs or converted from existing alignment formats; 
-	Is compact in file size; 
-	Allows most of operations on the alignment to work on a stream without loading the whole alignment into memory; 
-	Allows the file to be indexed by genomic position to efficiently retrieve all reads aligning to a locus. 

####VEP:			http://www.ensembl.org/Tools/VEP
The VEP determines the effect of your variants (SNPs, insertions, deletions, CNVs or structural variants) on genes, transcripts, and protein sequence, as well as regulatory regions. VEP reports:
-	genes and transcripts affected by the variants 
-	location of the variants (e.g. upstream of a transcript, in coding sequence, in non-coding RNA, in regulatory regions) 
-	consequence of your variants on the protein sequence (e.g. stop gained, missense, stop lost, frameshift) 
-	known variants that match yours, and associated minor allele frequencies from the 1000 Genomes Project 
-	SIFT and PolyPhen scores for changes to protein sequence
